In [1]:
! python -m numpy.f2py -c Kirchoff_sub_rnt.f90 -m kirchhoff2

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "kirchhoff2" sources
f2py options: []
f2py:> /var/folders/q1/8h226kws6qj8hctylvlyybhh0000gn/T/tmpd_cyv_w9/src.macosx-10.7-x86_64-3.7/kirchhoff2module.c
creating /var/folders/q1/8h226kws6qj8hctylvlyybhh0000gn/T/tmpd_cyv_w9/src.macosx-10.7-x86_64-3.7
Reading fortran codes...
	Reading file 'Kirchoff_sub_rnt.f90' (format:free)
	Reading file '/opt/intel/compilers_and_libraries_2020.1.216/mac/mkl/include/mkl_dss.fi' (format:fix)
Line #174 in Kirchoff_sub_rnt.f90:"        intent(in,out), DIMENSION(mtot+1) :: pot"
	analyzeline: cannot handle multiple attributes without type specification. Ignoring ' dimension(mtot+1)'.
Post-processing...
	Block: kirchhoff2
			Block: kirchhoff
				Block: dss_factor
				Block: dss_factor_real
				Bloc

/usr/local/bin/gfortran -Wall -g -m64 -Wall -g -undefined dynamic_lookup -bundle /var/folders/q1/8h226kws6qj8hctylvlyybhh0000gn/T/tmpd_cyv_w9/var/folders/q1/8h226kws6qj8hctylvlyybhh0000gn/T/tmpd_cyv_w9/src.macosx-10.7-x86_64-3.7/kirchhoff2module.o /var/folders/q1/8h226kws6qj8hctylvlyybhh0000gn/T/tmpd_cyv_w9/var/folders/q1/8h226kws6qj8hctylvlyybhh0000gn/T/tmpd_cyv_w9/src.macosx-10.7-x86_64-3.7/fortranobject.o /var/folders/q1/8h226kws6qj8hctylvlyybhh0000gn/T/tmpd_cyv_w9/Kirchoff_sub_rnt.o -L/usr/local/gfortran/lib/gcc/x86_64-apple-darwin16/6.3.0 -L/usr/local/gfortran/lib/gcc/x86_64-apple-darwin16/6.3.0/../../.. -L/usr/local/gfortran/lib/gcc/x86_64-apple-darwin16/6.3.0/../../.. -lgfortran -o ./kirchhoff2.cpython-37m-darwin.so
Removing build directory /var/folders/q1/8h226kws6qj8hctylvlyybhh0000gn/T/tmpd_cyv_w9


In [2]:
import mkl
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import kirchhoff2
import time
import scipy.linalg.lapack as la
from scipy.linalg import solve
from scipy.sparse import csr_matrix 
import scipy.sparse.linalg as spla


In [3]:
# #### uncomment to veiw the details of fortran subroutines used ####
print(kirchhoff2.kirchhoff.__doc__)
print(kirchhoff2.efield.__doc__)

pot,iext,resist,gmat = kirchhoff(gamma,ms,rload,volt,mtot,[mx,my])

Wrapper for ``kirchhoff``.

Parameters
----------
gamma : input float
ms : input rank-2 array('d') with bounds (mx,my)
rload : input float
volt : input float
mtot : input int

Other Parameters
----------------
mx : input int, optional
    Default: shape(ms,0)
my : input int, optional
    Default: shape(ms,1)

Returns
-------
pot : rank-1 array('d') with bounds (mtot + 1)
iext : rank-1 array('d') with bounds (mtot + 1)
resist : rank-2 array('d') with bounds (mx,my)
gmat : rank-2 array('d') with bounds (mtot + 1,mtot + 1)

pot,ex,ey = efield(dx,dy,pot,mx,my,[mtot])

Wrapper for ``efield``.

Parameters
----------
dx : input float
dy : input float
pot : input rank-1 array('d') with bounds (mtot + 1)
mx : input int
my : input int

Other Parameters
----------------
mtot : input int, optional
    Default: (len(pot)-1)

Returns
-------
pot : rank-1 array('d') with bounds (mtot + 1)
ex : rank-2 array('d') with bounds (mx,my)
ey

In [4]:
mx = 10
my = 10
mtot = mx*my
coef = 0.1
gamma = 0.05 
Rload = 0.5
dE = 0.1
dx = 1.0
dy = 1.0
Ly = dy*(float(my)-1.0) 
volt = dE*Ly
ms = np.ones((mx,my))
# ms = np.random.random((mx,my))
mfphi = True
print(volt)

0.9


In [5]:
def field(f_in):
    f = np.reshape(f_in[0:mtot],(mx,my))  
    dux = np.zeros((mx,my))
    duy = np.zeros((mx,my))

    duy[1:-1,:] = 0.5*(f[2:,:] - f[0:-2,:])/dy
    duy[0,:] = (f[1,:] - f[0,:])/dy
    duy[-1,:] = (f[-1,:] - f[-2,:])/dy
    duy = np.transpose(duy)

    dux[:,1:-1] = 0.5*(f[:,2:] - f[:,:-2])/dx
#     dux[:,0] = 0.5*(f[:,1] - f[:,-1])/dx
#     dux[:,-1] = 0.5*(-f[:,-2] + f[:,0] )/dx
    dux[:,0] = 0.5*(f[:,1] - f[:,0])/dx
    dux[:,-1] = 0.5*(-f[:,-2] + f[:,-1] )/dx    
    dux = np.transpose(dux)
    return dux, duy



# resist = gamma*np.maximum(np.ones((mx,my)),(ms/gamma)**2)
# gmat = np.zeros((mtot+1,mtot+1))
# print(resist)
def kirch_py(gamma, ms, Rload, volt, mtot):
    iext = np.zeros(mtot+1)
    gmat = np.zeros((mtot+1,mtot+1))
#     pot = np.zeros(mtot)
    resist = gamma*np.maximum(np.ones((mx,my)),(ms/gamma)**2)

    for i in range(0,mx):
        for j in range(0,my): 
            gmat[mx*j+(i+1)%mx, mx*j+(i)]       += - 2.0/(resist[i,j]+resist[(i+1)%mx,j])
            gmat[mx*j+(i),      mx*j+(i+1)%mx]  += - 2.0/(resist[i,j]+resist[(i+1)%mx,j])
            gmat[mx*j+(i),      mx*j+(i)]       += 2.0/(resist[i,j]+resist[(i+1)%mx,j])
            gmat[mx*j+(i+1)%mx, mx*j+(i+1)%mx]  += 2.0/(resist[i,j]+resist[(i+1)%mx,j])
            
        for j in range(0,my-1):
            gmat[mx*(j+1)+i, mx*(j)+i]   += - 2.0/(resist[i,j]+resist[i,j+1])
            gmat[mx*(j)+i,   mx*(j+1)+i] += - 2.0/(resist[i,j]+resist[i,j+1])
            gmat[mx*(j)+i,   mx*(j)+i]   += 2.0/(resist[i,j]+resist[i,j+1])
            gmat[mx*(j+1)+i, mx*(j+1)+i] += 2.0/(resist[i,j]+resist[i,j+1])
           
        gmat[i,i] += 1.0/resist[i,0]
        iext[i] += volt*(1.0/resist[i,0])

        gmat[mx*(my-1)+i,mx*(my-1)+i] += 1.0/resist[i,my-1]
        gmat[mtot,mtot]               += 1.0/resist[i,my-1]
        gmat[mx*(my-1)+i,mtot] += - 1.0/resist[i,my-1]
        gmat[mtot,mx*(my-1)+i] += - 1.0/resist[i,my-1]
    
    gmat[mtot,mtot] += 1.0/Rload

    A = csr_matrix(gmat)
#     print(A.csr_matrix.nnz())
    pot = spla.spsolve(A,iext)
#     pot = spla.dsolve.spsolve(gmat,iext,use_umfpack=True)
#     A, pot, Cons = la.dposv(gmat,iext,1,1)
#     pot = solve(gmat,iext) 
    dux, duy = field(pot)   

    return resist, gmat, pot, dux, duy







In [6]:
for i in range(2):
#     ms = np.random.random((mx,my))
    t0 = time.time()
    rr , gmat, pt, ex, ey = kirch_py(gamma, ms, Rload, volt, mtot)
#     print(pt)
    print(time.time() - t0)

# print(gmat)
# print(rr)
# # print(iext)
print(pt)


ex, ey = field(pt)
print(ex)
print(ey)

0.00511622428894043
0.00199127197265625
[0.82 0.82 0.82 0.82 0.82 0.82 0.82 0.82 0.82 0.82 0.74 0.74 0.74 0.74
 0.74 0.74 0.74 0.74 0.74 0.74 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66
 0.66 0.66 0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.42 0.42 0.42 0.42 0.42 0.42
 0.42 0.42 0.42 0.42 0.34 0.34 0.34 0.34 0.34 0.34 0.34 0.34 0.34 0.34
 0.26 0.26 0.26 0.26 0.26 0.26 0.26 0.26 0.26 0.26 0.18 0.18 0.18 0.18
 0.18 0.18 0.18 0.18 0.18 0.18 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1
 0.1  0.1  0.02]
[[ 5.55111512e-17  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.11022302e-16  0.00000000e+00  0.00000000e+00
   2.77555756e-17  0.00000000e+00]
 [ 0.00000000e+00  5.55111512e-17  1.11022302e-16  5.55111512e-17
   5.55111512e-17  1.11022302e-16  5.55111512e-17  2.77555756e-17
   5.55111512e-17  6.93889390e-18]
 [-1.66533454e-16 -5.55111512e-17  1.11022302e-16  5.55111512e-17
   0.00000000e+00 -5.55111512e-17  2.77555756e-1

In [7]:
# for i in range(2):
# #     ms = np.random.random((mx,my))
#     t0 = time.time()
#     pot = kirchhoff2.kirchhoff(gamma,ms,Rload,volt,mtot,mx,my)
# #     print(pot)
#     print(time.time() - t0)


In [8]:
nx = 4
ny = 4
nt = nx*ny
resist = np.ones((nx,ny))

g2 = np.zeros((nt+1,nt+1))
for i in range(0,nx):
    for j in range(0,ny):
        g2[nx*j+(i+1)%nx, nx*j+(i)]       += - 2.0/(resist[i,j]+resist[(i+1)%nx,j])
        g2[nx*j+(i),      nx*j+(i+1)%nx]  += - 2.0/(resist[i,j]+resist[(i+1)%nx,j])
        g2[nx*j+(i),      nx*j+(i)]       += 2.0/(resist[i,j]+resist[(i+1)%nx,j])
        g2[nx*j+(i+1)%nx, nx*j+(i+1)%nx]  += 2.0/(resist[i,j]+resist[(i+1)%nx,j])
    for j in range(0,ny-1):
        g2[nx*(j+1)+i, nx*(j)+i]   += - 2.0/(resist[i,j]+resist[i,j+1])
        g2[nx*(j)+i,   nx*(j+1)+i] += - 2.0/(resist[i,j]+resist[i,j+1])
        g2[nx*(j)+i,   nx*(j)+i]   += 2.0/(resist[i,j]+resist[i,j+1])
        g2[nx*(j+1)+i, nx*(j+1)+i] += 2.0/(resist[i,j]+resist[i,j+1])
           
    g2[i,i] += 1.0/resist[i,0]
    
    g2[nx*(ny-1)+i,nx*(ny-1)+i] += 1.0/resist[i,ny-1]
    g2[nt,nt]               += 1.0/resist[i,ny-1]
    g2[nx*(ny-1)+i,nt] += - 1.0/resist[i,ny-1]
    g2[nt,nx*(ny-1)+i] += - 1.0/resist[i,ny-1]


print(g2)


[[ 4. -1.  0. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  4. -1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  4. -1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0. -1.  4.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  4. -1.  0. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0. -1.  4. -1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0. -1.  4. -1.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1. -1.  0. -1.  4.  0.  0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.  0.  4. -1.  0. -1. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0. -1.  4. -1.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0. -1.  4. -1.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1. -1.  0. -1.  4.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  4. -1.  0. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0. -1.  4. -1.  0. -1.]
 [ 0. 

In [9]:
pot = np.zeros(mtot)
pot,iext,resist,gmat = kirchhoff2.kirchhoff(gamma, ms, Rload, volt, mtot, mx, my)
print(gmat)
print(resist)

[[ 0.2  -0.05  0.   ...  0.    0.    0.  ]
 [-0.05  0.2  -0.05 ...  0.    0.    0.  ]
 [ 0.   -0.05  0.2  ...  0.    0.    0.  ]
 ...
 [ 0.    0.    0.   ...  0.2  -0.05 -0.05]
 [ 0.    0.    0.   ... -0.05  0.2  -0.05]
 [ 0.    0.    0.   ... -0.05 -0.05  2.5 ]]
[[20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]]


In [23]:
pt = np.arange(10.1,0.0,-0.1)
print(np.shape(pt))

Ex , Ey = field(pt)
print(Ex)
print(Ey)

pt, ex , ey = kirchhoff2.efield(dx,dy,pt,mx,my,mtot)
print(ex)
print(ey)

(101,)
[[-0.05 -0.05 -0.05 -0.05 -0.05 -0.05 -0.05 -0.05 -0.05 -0.05]
 [-0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1 ]
 [-0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1 ]
 [-0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1 ]
 [-0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1 ]
 [-0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1 ]
 [-0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1 ]
 [-0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1 ]
 [-0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1 ]
 [-0.05 -0.05 -0.05 -0.05 -0.05 -0.05 -0.05 -0.05 -0.05 -0.05]]
[[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. 

In [11]:
# Make the plots a bit bigger to see
# NOTE: Must be done in a separate cell
plt.rcParams['figure.dpi'] = 100

In [12]:
def gradient(f,dx,dy):
    du = np.zeros(np.shape(f))
    du[1:-1,0:] += ( (f[2:,0:] - f[1:-1,0:]) /dx)**2 + ((f[0:-2,0:] - f[1:-1,0:])/dx)**2 
    du[:,1:-1] += ((f[:,2:] - f[:,1:-1])/dy)**2 + ((f[:,0:-2] - f[:,1:-1])/dy)**2 
    du[0,:] += ((f[1,:] - f[0,:])/dx)**2 + ((f[-1,:] - f[0,:])/dx)**2
    du[-1,:] += ((f[0,0:] - f[-1,0:])/dx)**2 + ((f[-2,0:] - f[-1,0:])/dx)**2
    du[:,0] += ((f[:,1] - f[:,0])/dy)**2
    du[:,-1] += ((f[:,-2] - f[:,-1])/dy)**2
    return du


def free_energy(phi,Tc,Tb,E,r0,g2,g4,g6):
    term1 = -(1./2.)*g2*phi**2 + (1./2.)*g2*setTeff(Tb,E,phi)/Tc*phi**2
    term2 = 0.5*r0*gradient(phi,1.0,1.0)
    term3 = (1./4.)*g4*phi**4
    term4 = (1./6.)*g6*phi**6
    fe = term1 + term2 + term3 + term4
    return fe

# def setTeff(Tb,E,phi):
#     tbath = Tb
#     gam = 0.05
#     efield = 0.1*E
#     return np.sqrt(tbath**2 + efield**2/(gam**2 + phi**2) )

def setTeff(Tb, E, phi):
    return Tb

In [13]:
# # phi_val = np.random.uniform(-3.0,3.0,(20,20))
# Tc = 1.
# g2 = 1.0
# g4 = -1.0
# g6 = 0.2
# r0 = 5.0
# # np.sort(phi_val)
# phi_val = np.linspace(-3.,3.,100)
# phi_val = np.tile(phi_val,(100,100))
# # phi_grad_val = np.gradient(phi_val)
# tb = 1.105
# E = 0.1
# Eval = np.arange(4.,44.,4.)
# # Tval = np.arange(0.2,2.4,0.2)
# Tval = [1.3]
# # E = 0.0
# # print(phi_val)
# for tb in Tval:
#     f = free_energy(phi_val,Tc,tb,E,r0,g2,g4,g6)
#     plt.plot(phi_val[0,:],f[0,:],label=('tb=', tb))
    
# plt.legend()    
# plt.ylabel("fe")
# plt.xlabel(r'$\Delta$')
# plt.ylim(-5.,5.)
# plt.xlim(-3.,3.)
# plt.show()    

In [14]:
# X = [1,2,3,4,5,6,7]
# Y = [x**2 for x in X]
# g = np.zeros((2,len(X)))
# gg = np.zeros((10000,len(X)))
# # g[:,] = np.array([np.random.random()**2 for x in X])
# # X = np.array(X)
# ss = time.time()
# gg[:,:] = [np.random.random((len(X))) for i in range(10000)]
# print(time.time() - ss)
# print(type(gg))
# # ss = time.time()
# # gg[:,:] = [np.random.random((len(X))) for i in range(1000)]
# # print(time.time() - ss)

# ss = time.time()
# for i in range(10000):
#     gg[i,:] = np.random.random((len(X)))
# print(time.time() - ss)
# print(type(gg))
# ss = time.time()
# gg = np.array([np.random.random((len(X))) for i in range(10000)])
# print(time.time() - ss)



# print(X)
# print(Y)
# g = np.concatenate((gg,gg),axis=0)
# g = np.concatenate((g,gg),axis=0)
# print(np.shape(g))


In [15]:
type(np.random.random())


float

In [16]:
# row = np.array([0, 0,1,  2, 2, 2])
# col = np.array([0, 2, 2, 0, 1, 2])
# data = np.array([1, 2, 3, 4, 5, 6])
# c = csr_matrix((data, (row, col)), shape=(3, 3)).toarray()
# print(c)

In [17]:
# np.show_config()

In [18]:
# mkl.get_version()

In [19]:
# nmeas = np.array([1,2,4,8,16,32,64,128,256,512,1024,2048])
# t_f = np.array([0.49,0.69,1.,1.76,3.2,6.3,11.68,22.9,46.2,89.8,179.7,395.08])
# t_p = np.array([0.62,0.95,1.5,2.81,5.09,10.1,18.8,37.04,77.04,151.9,320.6,642.6])
# ratio = t_p/t_f
# print(t_p/t_f)
# lnm = np.log2(nmeas)

# # plt.scatter(lnm,t_f)
# # plt.scatter(lnm,t_p)
# # plt.xlabel("$N_{meas}$")
# plt.scatter(lnm,ratio)
# plt.xlabel("$log_2(N_{meas})$")
# plt.ylabel("time")
# plt.show()

In [20]:
# s1 = 'aaa'
# f = 24.0/7.

# ss = s1 + '_' + format(f, '.2f') 
# print(ss)

In [21]:
# run_par = {
#     ### for voltage loop
#     'dE' : 0.2,      # Increment steps for Electric Field
#     'minE' : 0.2,    # Min value of Electric Field
#     'maxE' : 22.2,    # Max value of Electric Field
#     ### for temperature loop
#     'dTb' : 0.02,
#     'minT' : 0.02,
#     'maxT' : 1.52,
#     'nmeas' : 2048,   # Number of measurements
#     'Tbath' : 0.1,   # Bath temperature
#     'seed' : 442423  # Random number generator seed
#     # 'NPEs' : 1       # Number of Processors for MPI ( = 1 if MPI is not used )
# }


# fl = open("some.dat","w")   # Storing input and run parameters
# for iparam,param in run_par.items():
#     fl.write( iparam + str(run_par[iparam]) )
# fl.close()

In [22]:
# A = np.ones(10)
# B = np.tile(A,(10,12))
# print(A)
# print(B)